## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

https://dbs.uni-leipzig.de/research/projects/object_matching/benchmark_datasets_for_entity_resolution

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1363 + 3226
clusters_total = 1300

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads//Amazon-GoogleProducts/Amazon.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['name'] = row.pop('title')
            row['source'] = 'google'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/GoogleProducts.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'amazon'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Amazon-GoogleProducts/Amzon_GoogleProducts_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAmazon']]
        id_right = id_enumerator[row['idGoogleBase']]
        true_pair_set.add(tuple(sorted([id_left, id_right])))

len(true_pair_set)

1300

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2404

In [10]:
len(cluster_dict)

1105

In [11]:
# TODO: deal with this difference
# from entity_embed.data_utils.utils import cluster_dict_to_id_pairs

# assert len(true_pair_set - cluster_dict_to_id_pairs(cluster_dict)) == 0

In [12]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [13]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 938,
  'description': 'improve your typing skills today! typing instructor deluxe has a progressive design that has been developed for over 19 years. typing instructor deluxe can provide the right lessons tests strengthening exercises practice material and typing games for your skill level. you can even build your own personal typing plan to focus on specific areas you would like to improve. if you think learning has to be all hard work and no fun think again! for beginning to advanced typists kids to adults typing instructor deluxe will motivate you to improve your typing speed and accuracy using a travel theme and exciting typing challenges.educates entertains and motivates: choose from many typing plans or build your ownnavigate easily and choose your typing materialnew! dynamic learning methodsave your results and reports to track progresslearn voice-touch typing (dictation)3 unique travel themescolorful photos and musicten exciting games300+ magazine articles',
  'manufact

## Preprocess

In [14]:
attr_list = ['name', 'description', 'manufacturer', 'price']

In [15]:
import unidecode
from entity_embed.data_utils.one_hot_encoders import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [16]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [17]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [18]:
attr_info_dict = {
    'name': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'description': {
        'is_multitoken': True,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'manufacturer': {
        'is_multitoken': False,
        'tokenizer': default_tokenizer,
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    },
    'price': {
        'is_multitoken': False,
        'tokenizer': None,
        'alphabet': alphabet,  # compute
        'max_str_len': None,  # compute
    }
}

In [19]:
from entity_embed import MultiSigLinkageEmbed

train_cluster_len = 200
valid_cluster_len = 200
ann_k = 100
use_mask = True
model = MultiSigLinkageEmbed(
    # data kwargs
    row_dict=row_dict,
    attr_info_dict=attr_info_dict,
    cluster_attr=cluster_attr,
    pos_pair_batch_size=45,
    neg_pair_batch_size=1225,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed,
    # model kwargs
    use_mask=use_mask,
    ann_k=ann_k,
)

17:11:18 INFO:For attr='name', computing actual alphabet and max_str_len
17:11:18 INFO:For attr='name', using actual_max_str_len=26
17:11:18 INFO:For attr='description', computing actual alphabet and max_str_len
17:11:18 INFO:actual_max_str_len=29 must be pair to enable NN pooling. Updating to 30
17:11:18 INFO:For attr='description', using actual_max_str_len=30
17:11:18 INFO:For attr='manufacturer', computing actual alphabet and max_str_len
17:11:18 INFO:actual_max_str_len=47 must be pair to enable NN pooling. Updating to 48
17:11:18 INFO:For attr='manufacturer', using actual_max_str_len=48
17:11:18 INFO:For attr='price', computing actual alphabet and max_str_len
17:11:18 INFO:For attr='price', using actual_max_str_len=14


## Training

In [20]:
gpus = 1
max_epochs = 50
check_val_every_n_epoch = 1
early_stopping_monitor = 'valid_f1_at_0.9'
tb_log_dir = 'tb_logs'
tb_name = 'amzn-googl'

model.fit(
    gpus=gpus,
    max_epochs=max_epochs,
    check_val_every_n_epoch=check_val_every_n_epoch,
    early_stopping_monitor=early_stopping_monitor,
    tb_log_dir=tb_log_dir,
    tb_name=tb_name,
)

17:11:18 INFO:Fit model_sig_i=0, learning signature with attr_list=['name', 'description', 'manufacturer', 'price']
17:11:18 INFO:GPU available: True, used: True
17:11:18 INFO:TPU available: None, using: 0 TPU cores
17:11:18 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
17:11:18 INFO:Train pair count: 292
17:11:18 INFO:Valid pair count: 265
17:11:18 INFO:Test pair count: 998
17:11:20 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 4.3 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
4.3 M     Trainable params
0         Non-trainable params
4.3 M     Total params


17:13:35 INFO:Fit model_sig_i=1, learning signature with attr_list=['description', 'manufacturer', 'price']
17:13:35 INFO:GPU available: True, used: True
17:13:35 INFO:TPU available: None, using: 0 TPU cores
17:13:35 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
17:13:35 INFO:
  | Name        | Type           | Params
-----------------------------------------------
0 | blocker_net | BlockerNet     | 3.3 M 
1 | losser      | NTXentLoss     | 0     
2 | miner       | BatchHardMiner | 0     
-----------------------------------------------
3.3 M     Trainable params
0         Non-trainable params
3.3 M     Total params


In [21]:
for lt_module in model.lt_module_list:
    display(lt_module.get_signature_weights())

{'name': 1.0, 'description': 0.0, 'manufacturer': 0.0, 'price': 0.0}

{'description': 0.7225561141967773,
 'manufacturer': 0.07499327510595322,
 'price': 0.20245061814785004}

## Testing manually 

In [22]:
model.datamodule.setup(stage='test')

17:15:07 INFO:Train pair count: 292
17:15:07 INFO:Valid pair count: 265
17:15:07 INFO:Test pair count: 998


In [23]:
test_row_dict = model.datamodule.test_row_dict
test_multisig_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=model.datamodule.left_id_set,
    right_id_set=model.datamodule.right_id_set,
    batch_size=16
)
test_multisig_dict.keys()

dict_keys([('name', 'description', 'manufacturer', 'price'), ('description', 'manufacturer', 'price')])

In [24]:
test_true_pair_set = model.datamodule.test_true_pair_set
len(test_true_pair_set)

834

In [25]:
%%time

from entity_embed import MultiSigANNLinkageIndex

ann_index = MultiSigANNLinkageIndex(
    multisig_dict_keys=model.multisig_dict_keys,
    embedding_size=model.embedding_size,
)
ann_index.insert_multisig_dict(test_multisig_dict)
ann_index.build()

CPU times: user 716 ms, sys: 11.7 ms, total: 727 ms
Wall time: 109 ms


In [34]:
%%time

sim_threshold_dict = {
    ('name', 'description', 'manufacturer', 'price'): 0.3,
    ('description', 'manufacturer', 'price'): 0.3,
}
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold_dict=sim_threshold_dict,
    multisig_dict=test_multisig_dict,
)

CPU times: user 1.4 s, sys: 0 ns, total: 1.4 s
Wall time: 224 ms


In [35]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

37.5481144343303

In [36]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.01433791061317079, 0.9928057553956835)

In [37]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

56921

In [38]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

6

In [39]:
cos_similarity = lambda a, b: np.dot(a, b)

In [40]:
test_multisig_dict.keys()

dict_keys([('name', 'description', 'manufacturer', 'price'), ('description', 'manufacturer', 'price')])

In [41]:
test_left_vector_dict, test_right_vector_dict = \
    test_multisig_dict[('name', 'description', 'manufacturer', 'price')]

for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.116450086,
 {'id': 918,
  'description': 'mobi 70008 recam 4 . 6 camera monitoring and notification software surveillance software captures and analyzes images for advanced monitoring and recording of home or office ; program performs simultaneous',
  'manufacturer': 'mobi technologies inc .',
  'price': '129',
  'name': 'recam remote monitoring software',
  'source': 'google',
  'cluster_id': 918},
 {'id': 3921,
  'name': 'mobi - cam 70008 monitoring and notification software',
  'description': 'easily set - up and monitor your home or business observation system from anywhere notification of alarm and viewing via e - mail internet or cell phone multiple camera support',
  'manufacturer': 'mobi - cam',
  'price': '89 . 69',
  'source': 'amazon',
  'cluster_id': 918})

(-0.05042836,
 {'id': 845,
  'description': "iplaymusic ' s beginner guitar lessons is the first guitar learning program optimized for the mac and video ipod . the software takes advantage of ilife applications resident on today",
  'manufacturer': 'iplaymusic',
  'price': '49 . 99',
  'name': 'iplaymusic beginner guitar lessons for the mac and ipod',
  'source': 'google',
  'cluster_id': 845},
 {'id': 2324,
  'name': "wingnuts 2 : raina ' s revenge",
  'description': 'system requirements : mac os x 10 . 4 + g4 / g5 / intel 800 + mhz cpu 512 mb ram 32 mb video card 950 mb hard drive',
  'manufacturer': '',
  'price': '28 . 99',
  'source': 'amazon',
  'cluster_id': 845})

(0.15076198,
 {'id': 818,
  'description': "canopus ( 770 - 10158 - 100 ) let ' s edit 2 . 0",
  'manufacturer': 'canopus',
  'price': '99',
  'name': "canopus 77010158100 let ' s edit",
  'source': 'google',
  'cluster_id': 818},
 {'id': 4046,
  'name': "canopus let ' s edit - video editing software - win consumer video editing software",
  'description': "let ' s edit - realtime video editing software with movie - style effects - win canopus let ' s edit is fast and easy - to - use video",
  'manufacturer': '',
  'price': '99 . 95',
  'source': 'amazon',
  'cluster_id': 818})

(0.26292187,
 {'id': 165,
  'description': 'license - - 1 user ( s ) - standard - pc',
  'manufacturer': 'hewlett packard',
  'price': '0',
  'name': 'hp storageworks secure path for windows workgroup edition - ( v . 4 . 0c ) - license ( 213076 - b26 )',
  'source': 'google',
  'cluster_id': 165},
 {'id': 2866,
  'name': 'hewlett packard 213076 - b26 secure path v4 . 0c win wkgp ed 1 lic / cd',
  'description': 'secure path v4 . 0c win wkgp ed 1 lic / cd',
  'manufacturer': '',
  'price': '1888 . 39',
  'source': 'amazon',
  'cluster_id': 165})

(0.24829493,
 {'id': 506,
  'description': 'produce documents and office presentations that look great in multiple formats -- and much more ! / version upgrade enhance your personal documents with powerful graphics tools - add 3d',
  'manufacturer': 'corel',
  'price': '149 . 99',
  'name': 'corel wordperfect office 12 standard upgrade',
  'source': 'google',
  'cluster_id': 506},
 {'id': 1744,
  'name': 'corel corel word perfect office 12 upgrade',
  'description': 'super fast shipping - platinum reseller ratings - over 12 years in business wordperfect office 12 is a feature - rich compatible office productivity suite that offers powerful functionality at',
  'manufacturer': '',
  'price': '49 . 99',
  'source': 'amazon',
  'cluster_id': 506})

(0.21187639,
 {'id': 662,
  'description': 'terminal server lets you deliver windows - based applications or the windows desktop itself to virtually any computing device -- including those that cannot run windows . terminal server provides',
  'manufacturer': 'microsoft',
  'price': '669',
  'name': 'microsoft windows terminal server 2003 client additional license for users - 5 user',
  'source': 'google',
  'cluster_id': 662},
 {'id': 2908,
  'name': 'win 2003 ter svr cal 5pk microsoft r19 - 00846',
  'description': 'model - ms28736nt vendor - microsoft features - windows terminal svr cal 2003 english microsoft license pack user 5 pack calmanufacturer warranty : 90 days',
  'manufacturer': '',
  'price': '762 . 95',
  'source': 'amazon',
  'cluster_id': 662})

In [ ]:
test_left_vector_dict, test_right_vector_dict = \
    test_multisig_dict[('manufacturer', 'price')]

for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )